In [10]:
import basix
from basix import ElementFamily, CellType, LagrangeVariant
import jax.numpy as jnp
import numpy as np
import jax
import h5py
from fenics import *
import _ctypes

In [14]:
def read_mesh_and_function(file_name, var_name):

    # Open solution file
    infile = XDMFFile(file_name + ".xdmf")
    infile_h5 = h5py.File(file_name + ".h5", "r")
    t_steps = len(infile_h5[var_name].keys())

    # Read in mesh
    mesh = Mesh()
    infile.read(mesh)

    # Read function
    V = FunctionSpace(mesh, "CG", 1)
    u = Function(V)
    solution = np.zeros((V.dim(), t_steps))
    for i in range(t_steps):
        infile.read_checkpoint(u, var_name, i - t_steps + 1)
        solution[:, i] = u.vector().get_local()

    # Clean up
    infile.close()
    infile_h5.close()

    return mesh, solution
    
nu = 0.1
mesh, u_ref = read_mesh_and_function("output/burgers_1D/nu_"+str(nu)+"/FOM", "u")

In [15]:
def di(obj_id):
    """ Inverse of id() function. """
    return _ctypes.PyObj_FromPtr(obj_id)

In [21]:
D = [0, 1, 2, 2, 1]
D_ref = [id(D[0]), id(D[1]), id(D[2]), id(D[3]), id(D[4])]

In [ ]:
L = jnp.array([5,2,3])

def foo(x):
    return L[di(x)]

results = jax.vmap(foo)(np.array(D_ref))
print(results)